In [4]:
import requests
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import csv
import numpy as np



In [2]:
#reads in the csv
csv_file = "../data/summer.csv"
directory_db = pd.read_csv(csv_file)

directory_db

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,"HERSCHMANN, Otto",AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,"DRIVAS, Dimitrios",GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,"MALOKINIS, Ioannis",GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,"CHASAPIS, Spiridon",GRE,Men,100M Freestyle For Sailors,Silver
...,...,...,...,...,...,...,...,...,...
31160,2012,London,Wrestling,Wrestling Freestyle,"JANIKOWSKI, Damian",POL,Men,Wg 84 KG,Bronze
31161,2012,London,Wrestling,Wrestling Freestyle,"REZAEI, Ghasem Gholamreza",IRI,Men,Wg 96 KG,Gold
31162,2012,London,Wrestling,Wrestling Freestyle,"TOTROV, Rustam",RUS,Men,Wg 96 KG,Silver
31163,2012,London,Wrestling,Wrestling Freestyle,"ALEKSANYAN, Artur",ARM,Men,Wg 96 KG,Bronze


In [40]:
#groups country by Country and Year
country_medal_db=directory_db.groupby(['Country','Year','City']).count()
country_medal_db=country_medal_db[['Medal']]
country_medal_db.reset_index(inplace=True)

#rearranges by year
country_medal_db.sort_values('Year', ascending=True,inplace=True)
country_medal_db.set_index(['Year','Country'],inplace=True)


country_medal_db

City  Medal
Year Country               
1896 AUT      Athens      5
     GBR      Athens      7
     GER      Athens     33
     DEN      Athens      6
     GRE      Athens     52
...              ...    ...
2012 GAB      London      1
     FRA      London     82
     RUS      London    130
     GER      London     94
     QAT      London      2

[1157 rows x 2 columns]

In [41]:
#calculates Medal accumalitve sum - though I didn't end up using it
country_medal_db['Medal_Sum'] = country_medal_db.groupby('Country').cumsum()
country_medal_db

City  Medal  Medal_Sum
Year Country                          
1896 AUT      Athens      5          5
     GBR      Athens      7          7
     GER      Athens     33         33
     DEN      Athens      6          6
     GRE      Athens     52         52
...              ...    ...        ...
2012 GAB      London      1          1
     FRA      London     82       1396
     RUS      London    130        768
     GER      London     94       1305
     QAT      London      2          4

[1157 rows x 3 columns]

In [42]:
#exports csv
country_medal_db.to_csv('../Data/summer_country_medals.csv',index=True)

In [35]:
#groups country by Country and Year
olympic_summer_db=directory_db.groupby(['City','Year']).count()

#country_medal_db=country_medal_db[['Medal']]
olympic_summer_db.reset_index(inplace=True)
olympic_summer_db=olympic_summer_db[['City','Year']]
#rearranges by year
olympic_summer_db.sort_values('Year', ascending=True,inplace=True)
olympic_summer_db.set_index(['Year'],inplace=True)


olympic_summer_db.to_csv('../Data/summer_game_cities.csv',index=True)
olympic_summer_db

,City
Year,
1896,Athens
1900,Paris
1904,St Louis
1908,London
1912,Stockholm
1920,Antwerp
1924,Paris
1928,Amsterdam
1932,Los Angeles


------------------------------Start SQL File Here-----------------------------

In [5]:
database_path= "../data/Olympics.sqlite"

In [6]:
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [9]:
inspector = inspect(conn)

In [10]:
inspector.get_table_names()

['countryData', 'hostsData', 'olympicsData']

In [197]:
data=pd.read_sql("SELECT * FROM olympicsData",conn)

In [199]:
my_data=data.groupby(["NOC","Year","Season","City","Sport","Medal","Sex"]).count()
my_data

ID  Name  Age  Height  \
NOC Year Season City        Sport     Medal  Sex                          
AFG 2008 Summer Beijing     Taekwondo Bronze M     1     1    1       1   
    2012 Summer London      Taekwondo Bronze M     1     1    1       1   
AHO 1988 Summer Seoul       Sailing   Silver M     1     1    1       1   
ALG 1984 Summer Los Angeles Boxing    Bronze M     2     2    2       2   
    1992 Summer Barcelona   Athletics Gold   F     1     1    1       1   
...                                               ..   ...  ...     ...   
ZIM 2004 Summer Athina      Swimming  Bronze F     1     1    1       1   
                                      Gold   F     1     1    1       1   
                                      Silver F     1     1    1       1   
    2008 Summer Beijing     Swimming  Gold   F     1     1    1       1   
                                      Silver F     3     3    3       3   

                                                  Weight  Team  Games  Event  
NOC Year Season City        Sport     Medal  Sex                              
AFG 2008 Summer Beijing     Taekwondo Bronze M         1     1      1      1  
    2012 Summer London      Taekwondo Bronze M         1     1      1      1  
AHO 1988 Summer Seoul       Sailing   Silver M         1     1      1      1  
ALG 1984 Summer Los Angeles Boxing    Bronze M         2     2      2      2  
    1992 Summer Barcelona   Athletics Gold   F         1     1      1      1  
...                                                  ...   ...    ...    ...  
ZIM 2004 Summer Athina      Swimming  Bronze F         1     1      1      1  
                                      Gold   F         1     1      1      1  
                                      Silver F         1     1      1      1  
    2008 Summer Beijing     Swimming  Gold   F         1     1      1      1  
                                      Silver F         3     3      3      3  

[12720 rows x 8 columns]

In [200]:
my_data=my_data[['Name']]
my_data.reset_index(inplace=True)

In [201]:
my_data.sort_values(['Year','NOC'], ascending=(True,True),inplace=True)
my_data

,NOC,Year,Season,City,Sport,Medal,Sex,Name
108,AUS,1896,Summer,Athina,Athletics,Gold,M,2
109,AUS,1896,Summer,Athina,Tennis,Bronze,M,1
470,AUT,1896,Summer,Athina,Cycling,Bronze,M,2
471,AUT,1896,Summer,Athina,Cycling,Gold,M,1
472,AUT,1896,Summer,Athina,Swimming,Gold,M,1
...,...,...,...,...,...,...,...,...
12624,VEN,2016,Summer,Rio de Janeiro,Athletics,Silver,F,1
12625,VEN,2016,Summer,Rio de Janeiro,Boxing,Bronze,M,1
12626,VEN,2016,Summer,Rio de Janeiro,Cycling,Bronze,F,1
12629,VIE,2016,Summer,Rio de Janeiro,Shooting,Gold,M,1


In [202]:
my_data.rename(columns={"Name":"Medals_Won"},inplace=True)

In [205]:
my_data

,NOC,Year,Season,City,Sport,Medal,Sex,Medals_Won
108,AUS,1896,Summer,Athina,Athletics,Gold,M,2
109,AUS,1896,Summer,Athina,Tennis,Bronze,M,1
470,AUT,1896,Summer,Athina,Cycling,Bronze,M,2
471,AUT,1896,Summer,Athina,Cycling,Gold,M,1
472,AUT,1896,Summer,Athina,Swimming,Gold,M,1
...,...,...,...,...,...,...,...,...
12624,VEN,2016,Summer,Rio de Janeiro,Athletics,Silver,F,1
12625,VEN,2016,Summer,Rio de Janeiro,Boxing,Bronze,M,1
12626,VEN,2016,Summer,Rio de Janeiro,Cycling,Bronze,F,1
12629,VIE,2016,Summer,Rio de Janeiro,Shooting,Gold,M,1


In [263]:
#dictionary of ISO Countries - I had to hard codethis because it ISO abbreviations are different from standard

country_ISO_List = {
    "AFG": "Afghanistan",
    "AHO": "Netherlands Antilles",
    "ALG": "Algeria",
    "ANZ": "Austria",
    "ARG": "Argentina",
    "ARM": "Armenia",
    "AUS": "Australia",
    "AUT": "Austria",
    "AZE": "Azerbaijan",
    "BAH": "Bahamas",
    "BAR": "Barbados",
    "BDI": "Burundi",
    "BEL": "Belgium",
    "BER": "Bermuda",
    "BLR": "Belarus",
    "BOH": "Bohemia",
    "BOT": "Botswana",
    "BRA": "Brazil",
    "BRN": "Bahrain",
    "BUL": "Bulgaria",
    "CAN": "Canada",
    "CHI": "Chile",
    "CHN": "China",
    "CIV": "Côte d'Ivoire",
    "CMR": "Cameroon",
    "COL": "Colombia",
    "CRC": "Costa Rica",
    "CRO": "Croatia",
    "CUB": "Cuba",
    "CYP": "Cyprus",
    "CZE": "Czechia",
    "DEN": "Denmark",
    "DJI": "Djibouti",
    "DOM": "Dominican Republic",
    "ECU": "Ecuador",
    "EGY": "Egypt",
    "ERI": "Eritrea",
    "ESP": "Spain",
    "EST": "Estonia",
    "ETH": "Ethiopia",
    "EUN" : "Unified Team",
    "FIJ": "Fiji",
    "FIN": "Finland",
    "FRA": "France",
    "FRG": "Germany",    
    "GAB": "Gabon",
    "GBR": "Great Britain",
    "GDR": "East Germany",
    "GEO": "Georgia",
    "GER": "Germany",
    "GHA": "Ghana",
    "GRE": "Greece",
    "GRN": "Grenada",
    "GUA": "Guatemala",
    "GUY": "Guyana",
    "HAI": "Haiti",
    "HKG": "Hong Kong",
    "HUN": "Hungary",
    "INA": "Indonesia",
    "IND": "India",
    "IOA": "Individual",
    "IRI": "Iran",
    "IRL": "Ireland",
    "IRQ": "Iraq",
    "ISL": "Iceland",
    "ISR": "Israel",
    "ISV": "Virgin Islands",
    "ITA": "Italy",
    "JAM": "Jamaica",
    "JOR": "Jordan",
    "JPN": "Japan",
    "KAZ": "Kazakhstan",
    "KEN": "Kenya",
    "KGZ": "Kyrgyzstan",
    "KOR": "Korea",
    "KSA": "Saudi Arabia",
    "KUW": "Kuwait",
    "LAT": "Latvia",
    "LIB": "Lebanon",
    "LIE": "Liechtenstein",
    "LTU": "Lithuania",
    "LUX": "Luxembourg", 
    "MAR": "Morocco",
    "MAS": "Malaysia",
    "MDA": "Moldova",
    "MEX": "Mexico",
    "MGL": "Mongolia",
    "MKD": "Macedonia",
    "MNE": "Montenegro",
    "MON": "Monaco",
    "MOZ": "Mozambique",
    "MRI": "Mauritius",
    "NAM": "Namibia",
    "NED": "Netherlands",
    "NEP": "Nepal",
    "NGR": "Nigeria",
    "NIG": "Nicaragua",
    "NOR": "Norway",
    "NZL": "New Zealand",
    "PAK": "Pakistan",
    "PAN": "Panama",
    "PAR": "Paraguay",
    "PER": "Peru",
    "PHI": "Philippines",
    "POL": "Poland",
    "POR": "Portugal",
    "PRK": "North Korea",
    "PUR": "Puerto Rico",
    "QAT": "Qatar",
    "ROU": "Romania",
    "RSA": "South Africa",
    "RUS": "Russia",
    "SCG": "Serbia and Montenegro",
    "SEN": "Senegal",
    "SGP": "Singapore",
    "SLO": "Slovenia",
    "SRB": "Serbia",
    "SRI": "Sri Lanka",
    "SUD": "Sudan",
    "SUI": "Switzerland",
    "SUR": "Suriname",
    "SVK": "Slovakia",
    "SWE": "Sweden",
    "SYR": "Syrian Arab Republic",
    "TAN": "Tanzania",
    "TCH": "Czechoslovakia",
    "TGA": "Tonga",
    "THA": "Thailand",
    "TJK": "Tajikistan",
    "TOG": "Togo",
    "TPE": "Republic of China",
    "TTO": "Trinidad and Tobago","TUN": "Tunisia",
    "TUN": "Tunisia",
    "TUR": "Turkey",
    "UAE": "United Arab Emirates",
    "UAR": "Egypt",
    "UGA": "Uganda",
    "UKR": "Ukraine",
    "URS": "Soviet Union",
    "URU": "Uruguay",
    "USA": "United States",
    "UZB": "Uzbekistan",
    "VEN": "Venezuela",
    "VIE": "Vietnam",
    "WIF": "Wallis and Futuna",
    "YUG": "Yugoslavia",
    "ZAM": "Zambia",
    "ZIM": "Zimbabwe"
};

In [281]:
#set up db of coutnries
countryNames=pd.DataFrame({
    "NOC":list(country_ISO_List.keys()),
    "Name":list(country_ISO_List.values())
})


countryNames.head(5)


,NOC,Name
0,AFG,Afghanistan
1,AHO,Netherlands Antilles
2,ALG,Algeria
3,ANZ,Austria
4,ARG,Argentina


In [265]:
db_merged=pd.merge(my_data,countryNames,on="NOC",how="inner")

In [266]:
db_merged.rename(columns={"NOC":"Country"},inplace=True)

In [267]:
db_merged=db_merged[['Year','Season','City','Country', 'Name','Sex','Sport','Medal','Medals_Won']]

In [268]:
db_merged.sort_values(['Year','Country'],inplace=True)

In [269]:
#final db to pull into js
db_merged.head()

,Year,Season,City,Country,Name,Sex,Sport,Medal,Medals_Won
0,1896,Summer,Athina,AUS,Australia,M,Athletics,Gold,2
1,1896,Summer,Athina,AUS,Australia,M,Tennis,Bronze,1
362,1896,Summer,Athina,AUT,Austria,M,Cycling,Bronze,2
363,1896,Summer,Athina,AUT,Austria,M,Cycling,Gold,1
364,1896,Summer,Athina,AUT,Austria,M,Swimming,Gold,1


In [271]:
db_merged['Season'][0]

'Summer'